In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv('/kaggle/input/math-students/student-mat.csv')
data

In [ ]:
school=data.groupby('school')
school.first()

* Check for null values

In [ ]:
data.isna().sum()

In [ ]:
data.describe()

* The Average of G3 is 10.415.So, let's take students with Marks of above mean to pass and below fail
* Drop G1,G2 as G3 is the final grades and G1,G2 are related to G3

In [ ]:
data['Overall'] = np.where(data['G3'] >=10.415, 'Pass', 'Fail')
data.head()

In [ ]:
data.describe()

In [ ]:
fail_by_gender=data[(data['Overall']=='Fail')]['sex'].value_counts()
fail_by_gender.plot.bar()

In [ ]:
pass_by_gender=data[(data['Overall']=='Pass')]['sex'].value_counts()
pass_by_gender.plot.bar()

In [ ]:
columns_list=data.columns
data.romantic.head()

In [ ]:
import matplotlib.pyplot as plt
fail=data[(data['Overall']=='Fail') & (data['romantic']=='yes')]['sex'].value_counts()
passed=data[(data['Overall']=='Fail') & (data['romantic']=='no')]['sex'].value_counts()

fail,passed

In [ ]:
fail.plot.bar()

In [ ]:
passed.plot.bar()

* We have 209 passed students and 186 failed students , The dataset is somewhat unbalanced
* Now let's encode the columns with binary values

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


In [ ]:
cols=data.columns[data.dtypes==object].tolist()
for col in cols:
    n = len(data[col].unique())
    if (n == 2):
        data[col] = pd.get_dummies(data[col], drop_first=True)

In [ ]:
data=pd.get_dummies(data)
data=pd.get_dummies(data)
data

In [ ]:
check_for_school=data[data['Overall']=='Fail'].groupby('school')
check_for_school

In [ ]:
cols=data.columns.tolist()
cols

In [ ]:
new_corr=data.corr()
new_corr['G3'].sort_values(ascending=False)

**we can remove features that have very less impact on G3 or overall**
* i took the threshold of 0.08 correlation

In [ ]:
data=data.drop(['G1','G2','G3','activities','guardian_mother','guardian_father','guardian_other','famsup','reason_home','Fjob_services','Fjob_at_home'],axis=1)
data.head()

<h1>Observations Till Now</h1>
* *Mother education,Father education,hoping for higher studies have positive effect on grades
* *failures,alcohol,absences,romantic relationship,mother working from home, have very negative effect on final Grades 

<h2>Splitting data</h2>
* We can now split data , 1/3 for test and rest for training

In [ ]:
y=data.Overall
x=data.drop('Overall',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)
print("train:",len(x_train),"test:",len(x_test))

<h1> Applying Algorithms</h1>

***Linear Regression***

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
lr.fit(x_train, y_train)
predictions = lr.predict(x_test)
cm = confusion_matrix(y_test, predictions)
print(accuracy_score(y_test,predictions)*100)

**Random Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 50)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(accuracy_score(y_test,y_pred)*100)

<h1>Observations</h1>
* Absence from classes negatively affected the overall grade
* Relationship status affected grade negatively but effect more pronunced for Males than females
* Mother and father Education has very high positive impact on Grades
* Internet,Guardian,sex has very little impact
<br><br>
* Since, it is a correlated data Random forest classifier gave less accuracy than LogisticRegression  